In [20]:
import pandas as pd

def read_levels_transitions_HULLAC(
    atom_number,
    ion_charge,
    file_levels,
    file_transitions,
    only_levels=False,
):
    """
    Reads HULLAC-formatted energy levels and transitions into pandas DataFrames.

    Parameters
    ----------
    atom_number : int
        Atomic number of the element.
    ion_charge : int
        Ionization charge.
    file_levels : str
        Path to the levels file.
    file_transitions : str
        Path to the transitions file.
    only_levels : bool, optional
        If True, return only energy levels.

    Returns
    -------
    levels : pd.DataFrame
        Parsed energy levels.
    transitions : pd.DataFrame, optional
        Parsed transitions. Returned only if only_levels is False.
    """
    # Read energy levels
    raw_levels = pd.read_csv(
        file_levels,
        delimiter=r"\s+",
        names=["level_index", "2J+1", "parity", "energy"],
        skiprows=1,
    )
    raw_levels["j"] = (raw_levels["2J+1"].astype(float) - 1) / 2.0

    levels = raw_levels[["level_index", "energy", "j"]].copy()
    levels["energy"] = levels["energy"].astype(float) * 8065.544004795713  # Convert to cm^-1
    levels["level_index"] -= 1
    levels["atomic_number"] = atom_number
    levels["ion_charge"] = ion_charge
    levels["label"] = None
    levels["method"] = "calc HULLAC"
    levels["priority"] = 10

    levels = levels.set_index(["atomic_number", "ion_charge", "level_index"])

    if only_levels:
        return levels

    # Read transitions
    raw_transitions = pd.read_csv(
        file_transitions,
        delimiter=r"\s+",
        names=["level_index_upper", "level_index_lower", "wavelength", "g_upper_A", "loggf"],
        skiprows=1,
    )
    raw_transitions["gf"] = 10 ** raw_transitions["loggf"]

    transitions = raw_transitions[["level_index_upper", "level_index_lower", "gf", "wavelength"]].copy()
    transitions["level_index_upper"] -= 1
    transitions["level_index_lower"] -= 1
    transitions["atomic_number"] = atom_number
    transitions["ion_charge"] = ion_charge

    # Map j and energy values from levels
    level_j = levels["j"]
    level_energy = levels["energy"]

    transitions["j_lower"] = transitions["level_index_lower"].map(level_j)
    transitions["j_upper"] = transitions["level_index_upper"].map(level_j)
    transitions["energy_lower"] = transitions["level_index_lower"].map(level_energy)
    transitions["energy_upper"] = transitions["level_index_upper"].map(level_energy)

    # # Filter out short-wavelength transitions
    # before_filter = len(transitions)
    # transitions = transitions[transitions["wavelength"] > 150]
    # after_filter = len(transitions)
    # print(f"Filtered transitions: {before_filter} → {after_filter}")

    transitions = transitions.set_index(
        ["atomic_number", "ion_charge", "level_index_lower", "level_index_upper"]
    )
    transitions = transitions.sort_index()

    return levels, transitions


class HULLAC_Reader:
    """
    Class for extracting levels and lines from HULLAC data.

    Mimics the GFALLReader class.

    Attributes
    ----------
    levels : pd.DataFrame
        Energy levels.
    lines : pd.DataFrame
        Transitions.
    """

    def __init__(self, data, priority=10):
        """
        Parameters
        ----------
        data : dict
            Dictionary containing species-specific dictionaries with
            keys `levels` and `lines`.

        priority : int, optional
            Priority of the current data source, by default 10.
        """
        self.priority = priority
        self._get_levels_lines(data)
        self.version = None

    def _get_levels_lines(self, data):
        """
        Populates `self.levels` and `self.lines` from a data dictionary.

        Parameters
        ----------
        data : dict
            Dictionary containing one dictionary per species with
            keys `levels` and `lines`.
        """
        lvl_list = []
        lns_list = []

        for ion_data in data.values():
            levels, lines = read_levels_transitions_HULLAC(
                atom_number=ion_data['atom_number'],
                ion_charge=ion_data['ion_charge'],
                file_levels=ion_data['levels'],
                file_transitions=ion_data['lines']
            )
            lvl_list.append(levels)
            lns_list.append(lines)

        self.levels = pd.concat(lvl_list)
        self.lines = pd.concat(lns_list)


In [21]:
prefix_path = '/Users/afloers/Work/ATOMIC_DATA/HULLAC_all/'
la_1_lvl = prefix_path + '57LaII.lev.asc'
la_2_lvl = prefix_path + '57LaIII.lev.asc'

la_1_tr = prefix_path + '57LaII.tr.asc'
la_2_tr = prefix_path + '57LaIII.tr.asc'

hullac_atomic = {'La 1': {'atom_number': 57, 'ion_charge': 1, 'levels': la_1_lvl, 'lines': la_1_tr},
              'La 2': {'atom_number': 57, 'ion_charge': 2, 'levels': la_2_lvl, 'lines': la_2_tr},
             }
tanaka_reader = HULLAC_Reader(hullac_atomic)
                 

In [23]:
from carsus.io.cmfgen import CMFGENReader, CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser
import pandas as pd
from carsus.util import parse_selected_species
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData
from carsus.io.zeta import KnoxLongZeta

In [24]:
# tanaka_reader = CustomAtomicReader_Tanaka(tanaka_data, priority=20)


atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-U')


gfall_reader = GFALLReader('H-U')


chianti_reader = ChiantiReader('H-He', collisions=True, priority=20)



zeta_data = KnoxLongZeta()
display(tanaka_reader.levels)

atom_data_with_cmfgen = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           tanaka_reader)
atom_data_with_cmfgen.to_hdf('kurucz_tanaka_except_Sr_Y_Zr_Ba3.h5')

[ carsus.io.nist.weightscomp][   INFO] - Downloading data from the carsus-dat-nist repository (weightscomp.py:69)
[                py.warnings][WARNING] - /Users/afloers/Software/mambaforge/envs/carsus/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
 (warnings.py:112)
[  carsus.io.nist.ionization][   INFO] - Downloading ionization energies from the carsus-data-nist repo. (ionization.py:87)
[                py.warnings][WARNING] - /Users/afloers/Software/mambaforge/envs/carsus/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/late

energy    j label       method  \
atomic_number ion_charge level_index                                          
57            1          0                 0.000000  2.0  None  calc HULLAC   
                         1               904.466717  3.0  None  calc HULLAC   
                         2              1618.617326  2.0  None  calc HULLAC   
                         3              1813.925764  4.0  None  calc HULLAC   
                         4              2760.313308  1.0  None  calc HULLAC   
...                                             ...  ...   ...          ...   
              2          10            91749.773269  0.5  None  calc HULLAC   
                         11            91884.492050  2.5  None  calc HULLAC   
                         12            91917.262355  3.5  None  calc HULLAC   
                         13            92872.376011  1.5  None  calc HULLAC   
                         14           108209.128919  0.5  None  calc HULLAC   

                                      priority  
atomic_number ion_charge level_index            
57            1          0                  10  
                         1                  10  
                         2                  10  
                         3                  10  
                         4                  10  
...                                        ...  
              2          10                 10  
                         11                 10  
                         12                 10  
                         13                 10  
                         14                 10  

[81 rows x 5 columns]

[carsus.io.output.levels_lines][   INFO] - Ingesting energy levels. (levels_lines.py:155)
[     carsus.io.kurucz.gfall][   INFO] - Parsing GFALL from: https://github.com/tardis-sn/carsus-data-kurucz/raw/main/linelists/gfall/gfall.dat?raw=true (gfall.py:167)
[carsus.io.output.levels_lines][   INFO] - GFALL selected species: Li 0, Li 1, Be 0, Be 1, Be 2, B 0, B 1, B 2, B 3, C 0, C 1, C 2, C 3, N 0, N 1, N 2, N 3, N 4, N 5, O 0, O 1, O 2, O 3, O 4, O 5, F 0, F 1, F 2, F 3, F 4, F 5, Ne 0, Ne 1, Ne 2, Ne 3, Ne 4, Ne 5, Na 0, Na 1, Na 2, Na 3, Na 4, Na 5, Mg 0, Mg 1, Mg 2, Mg 3, Mg 4, Mg 5, Al 0, Al 1, Al 2, Al 3, Al 4, Al 5, Si 0, Si 1, Si 2, Si 3, Si 4, Si 5, P 0, P 1, P 2, P 3, P 4, P 5, S 0, S 1, S 2, S 3, S 4, S 5, Cl 0, Cl 1, Cl 2, Cl 3, Cl 4, Ar 0, Ar 1, Ar 2, Ar 3, Ar 4, K 0, K 1, K 2, K 3, K 4, Ca 0, Ca 1, Ca 2, Ca 3, Ca 4, Ca 5, Ca 6, Ca 7, Ca 8, Sc 0, Sc 1, Sc 2, Sc 3, Sc 4, Sc 5, Sc 6, Sc 7, Sc 8, Ti 0, Ti 1, Ti 2, Ti 3, Ti 4, Ti 5, Ti 6, Ti 7, Ti 8, V 0, V 1, V 2, V 3, V 4, V 5

In [16]:
a = atom_data_with_cmfgen.levels
a = a[a['atomic_number']==57]
a = a[a['ion_number']==1]
a

,level_id,atomic_number,ion_number,g,energy,metastable,level_number,ds_id
28511,42712,57,1,5,0.000000,True,0,5.0
28512,42713,57,1,7,0.112140,True,1,5.0
28513,42714,57,1,5,0.200683,True,2,5.0
28514,42715,57,1,9,0.224898,True,3,5.0
28515,42716,57,1,3,0.342235,True,4,5.0
...,...,...,...,...,...,...,...,...
28572,42773,57,1,7,4.766444,False,61,5.0
28573,42774,57,1,9,4.781597,False,62,5.0
28574,42775,57,1,7,4.922725,False,63,5.0
28575,42776,57,1,5,5.048675,False,64,5.0


In [17]:
b = atom_data_with_cmfgen.lines
b = b[b['atomic_number']==57]
a = b[b['ion_number']==1]
b

,line_id,lower_level_id,upper_level_id,wavelength,gf,loggf,A_ul,ds_id,atomic_number,ion_number,level_number_lower,g_l,level_number_upper,g_u,f_lu,f_ul,nu,B_lu,B_ul
280475,542333,33341,33458,2726.378270,0.707946,-0.150,7.941088e+07,2,57,0,1,6,118,8,0.117991,0.088493,1.099600e+15,5.400953e+09,4.050714e+09
280476,542334,33341,33453,3216.736919,0.199526,-0.700,1.607760e+07,2,57,0,1,6,113,8,0.033254,0.024941,9.319769e+14,1.795973e+09,1.346980e+09
280477,542335,33341,33445,3343.182035,0.416869,-0.380,3.109798e+07,2,57,0,1,6,105,8,0.069478,0.052109,8.967279e+14,3.899817e+09,2.924863e+09
280478,542336,33341,33441,3462.180457,0.239883,-0.620,3.337205e+07,2,57,0,1,6,101,4,0.039981,0.059971,8.659065e+14,2.323988e+09,3.485983e+09
280479,542337,33341,33437,3515.066032,0.147911,-0.830,1.330834e+07,2,57,0,1,6,97,6,0.024652,0.024652,8.528786e+14,1.454848e+09,1.454848e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301821,564180,42787,42789,9769.830000,0.183231,-0.737,2.134111e+06,5,57,2,9,6,11,6,0.030539,0.030539,3.068553e+14,5.009233e+09,5.009233e+09
301822,564181,42787,42790,9738.660000,3.689776,0.567,3.243798e+07,5,57,2,9,6,12,8,0.614963,0.461222,3.078375e+14,1.005503e+11,7.541274e+10
301823,564182,42787,42791,8909.900000,2.162719,0.335,4.542936e+07,5,57,2,9,6,13,4,0.360453,0.540680,3.364712e+14,5.392089e+10,8.088134e+10
301824,564183,42788,42792,6075.570000,0.615177,-0.211,5.558251e+07,5,57,2,10,2,14,2,0.307588,0.307588,4.934392e+14,3.137564e+10,3.137564e+10
